In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = " https://frenzy86.s3.eu-west-2.amazonaws.com/IFAO/boston_houses.csv"
data = pd.read_csv(url)

In [3]:
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [4]:
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
y = data['MEDV']

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [6]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

In [20]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [21]:
polyfeats = PolynomialFeatures(degree=2)
X_train_poly = polyfeats.fit_transform(X_train)
X_test_poly = polyfeats.transform(X_test)

print("Numero di esempi nel test: "+str(X_train_poly.shape[0]))
print("Numero di features: "+str(X_train_poly.shape[1]))

Numero di esempi nel test: 404
Numero di features: 105


In [22]:
scaler = StandardScaler()
X_train_poly = scaler.fit_transform(X_train_poly)
X_test_poly = scaler.transform(X_test_poly)

In [23]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
  
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

    print(f"\nRisultati per {model_name}:")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2 Score: {r2:.3f}")
    print(f"CV R2 Scores: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

    return y_pred, r2

In [24]:
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)
y_pred_elastic, r2_elastic = evaluate_model(elastic, X_train_poly, X_test_poly, y_train, y_test, "Elastic Net")


Risultati per Elastic Net:
RMSE: 3.74
R2 Score: 0.859
CV R2 Scores: 0.770 (+/- 0.029)


In [25]:
import joblib
joblib.dump(elastic,'elastik_model.pkl')


['elastik_model.pkl']

In [27]:
x_testnew = pd.DataFrame(X_test.iloc[0]).T
x_testnew

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
307,0.04932,33.0,2.18,0.0,0.472,6.849,70.3,3.1827,7.0,222.0,18.4,396.9,7.53


In [28]:

x_testnew=polyfeats.transform(x_testnew)
x_testnew= scaler.transform(x_testnew)

In [29]:
loaded_model = joblib.load ('elastik_model.pkl')
loaded_model.predict(x_testnew)[0]

31.062474180640994